In [20]:
# Initial imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import json

%matplotlib inline

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import MinMaxScaler
from keras.layers import *
from keras.optimizers import *
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs, make_moons

In [21]:
# Load .env environment variables
load_dotenv()

True

In [22]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY_ID")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

In [23]:
# Verify that Alpaca key and secret were correctly loaded
print(f"Alpaca Key type: {type(alpaca_api_key)}")
print(f"Alpaca Secret Key type: {type(alpaca_secret_key)}")

Alpaca Key type: <class 'str'>
Alpaca Secret Key type: <class 'str'>


In [24]:
# Create the Alpaca API object
api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)

In [25]:
# Set timeframe to '1D' pulling non-vol stock data
timeframe = "1D"

# Set start and end datetimes between now and 3 years ago.
start_date = pd.Timestamp("2019-01-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2021-06-01", tz="America/New_York").isoformat()
# Set the ticker informationkrak
tickers = ["WKHS","OPTI","LTRPB",]


In [26]:
groups = {"Small cap":["WKHS","OPTI","LTRPB"], "Mid cap":["RIOT","TTD","DXC"],"Large cap":["SQ","IBM","ENPH"]}

In [27]:
def window_data(df_ibm_ticker, window, feature_col_number, target_col_number):
    X = []
    y = []
    for i in range(len(df_ibm_ticker) - window - 1):
        features = df_ibm_ticker.iloc[i:(i + window), feature_col_number]
        target = df_ibm_ticker.iloc[(i + window), target_col_number]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).reshape(-1, 1)

In [32]:
#creating function to pull ticker names
def training_model(Current_ticker):
    df_ibm_ticker = api.get_barset(
       tickers,
       timeframe,
       start=start_date,
       end=end_date,
       limit=1000
    ).df
    # Display sample data
    df_ibm_ticker.head()
    
    #
    data = np.array(df_ibm_ticker)

    # train test split, we can take last 100 data points as test set
    train , test = data[0:-100], data[-100:]
    
   
        # Predict Closing Prices using a 10 day window of previous closing prices
    # Then, experiment with window sizes anywhere from 1 to 10 and see how the model performance changes
    window_size = 10

    # Column index 0 is the 'IBM ' column
    # Column index 1 is the `Close` column
    feature_column = 1
    target_column = 1
    X, y = window_data(df_ibm_ticker, window_size, feature_column, target_column)
    

        # Use 70% of the data for training and the remaineder for testing
    data_train = int(.7 * len(X))
    X_train = X[:data_train - 1]
    X_test = X[data_train:]
    y_train = y[:data_train - 1]
    y_test = y[data_train:]
    
        # Use the MinMaxScaler to scale data between 0 and 1.
    from sklearn.preprocessing import MinMaxScaler
    x_train_scaler = MinMaxScaler()
    x_test_scaler = MinMaxScaler()
    y_train_scaler = MinMaxScaler()
    y_test_scaler = MinMaxScaler()

    # Fit the scaler for the training Data
    x_train_scaler.fit(X_train)
    y_train_scaler.fit(y_train)

    # Scale the training data
    X_train = x_train_scaler.transform(X_train)
    y_train = y_train_scaler.transform(y_train)

    # Fit the scaler for the testing Data
    x_test_scaler.fit(X_test)
    y_test_scaler.fit(y_test)

    # Scale the y_test data
    X_test = x_test_scaler.transform(X_test)
    y_test = y_test_scaler.transform(y_test)

        # Reshape the features for the model
    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

    # Build the LSTM model. 
        # Note: The input shape is the number of time steps and the number of indicators
    # Note: Batching inputs has a different input shape of Samples/TimeSteps/Features
    model = Sequential()
    model.add(LSTM(
        units=30, return_sequences=True,
        input_shape=(X_train.shape[1], 1)))
    model.add(Dropout(0.2))
    model.add(LSTM(units=30, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(units=30))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    
        # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')

        # Summarize the model
    #model.summary()
    
        # Train the model
    # Use at least 10 epochs
    model.fit(X_train, y_train, epochs=5, shuffle=False, batch_size=1,
        verbose=1)
    
        # Evaluate the model
    model.evaluate(X_test, y_test)
    
            # Make some predictions
    predicted = model.predict(X_test)

        # Recover the original prices instead of the scaled version
    predicted_prices = y_test_scaler.inverse_transform(predicted)
    real_prices = y_test_scaler.inverse_transform(y_test.reshape(-1, 1))
    
        # Create a DataFrame of Real and Predicted values
    stocks = pd.DataFrame({
        "Real": real_prices.ravel(),
        "Predicted": predicted_prices.ravel()
    }, index = df_ibm_ticker.index[-len(real_prices): ]) 
    stocks.head()
    
    return stocks

In [34]:
training_model(["IBM"])

Epoch 1/5
416/416 [==============================] - 48s 42ms/step - loss: nan
Epoch 2/5
416/416 [==============================] - 22s 52ms/step - loss: nan
Epoch 3/5
416/416 [==============================] - 20s 49ms/step - loss: nan
Epoch 4/5
416/416 [==============================] - 17s 42ms/step - loss: nan
Epoch 5/5
416/416 [==============================] - 17s 41ms/step - loss: nan


,Real,Predicted
time,,
2020-09-14 04:00:00+00:00,39.5000,NaN
2020-09-15 04:00:00+00:00,39.0100,NaN
2020-09-16 04:00:00+00:00,41.9800,NaN
2020-09-17 04:00:00+00:00,38.6296,NaN
2020-09-18 04:00:00+00:00,38.2000,NaN
...,...,...
2021-05-25 04:00:00+00:00,30.5000,NaN
2021-05-26 04:00:00+00:00,36.0000,NaN
2021-05-27 04:00:00+00:00,34.5000,NaN


In [33]:
for(name,l) in groups.items():
    print (name)
    print (training_model(l))

Small cap
Epoch 1/5
416/416 [==============================] - 50s 37ms/step - loss: nan
Epoch 2/5
416/416 [==============================] - 14s 33ms/step - loss: nan
Epoch 3/5
416/416 [==============================] - 12s 29ms/step - loss: nan
Epoch 4/5
416/416 [==============================] - 16s 40ms/step - loss: nan0s - los
Epoch 5/5
416/416 [==============================] - 15s 36ms/step - loss: nan
                              Real  Predicted
time                                         
2020-09-14 04:00:00+00:00  39.5000        NaN
2020-09-15 04:00:00+00:00  39.0100        NaN
2020-09-16 04:00:00+00:00  41.9800        NaN
2020-09-17 04:00:00+00:00  38.6296        NaN
2020-09-18 04:00:00+00:00  38.2000        NaN
...                            ...        ...
2021-05-25 04:00:00+00:00  30.5000        NaN
2021-05-26 04:00:00+00:00  36.0000        NaN
2021-05-27 04:00:00+00:00  34.5000        NaN
2021-05-28 04:00:00+00:00  32.0000        NaN
2021-06-01 04:00:00+00:00  32.0500 